In [1]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd())); ROOT_DIR = '/home/h4zzkr/lab/delOS/backend'
import numpy as np
import pandas as pd
import re
datapath = os.path.join(ROOT_DIR, 'data/nlu_data/snipsai')

In [2]:
snips = pd.read_csv(os.path.join(datapath, 'train.csv'))
snips

,Unnamed: 0,intent_label,words,word_labels,length
0,0,AddToPlaylist,Add Don and Sherri to my Meditate to Sounds of...,O B-entity_name I-entity_name I-entity_name O ...,12
1,1,AddToPlaylist,put United Abominations onto my rare groove pl...,O B-entity_name I-entity_name O B-playlist_own...,8
2,2,AddToPlaylist,add the tune by misato watanabe to the Trapeo ...,O O B-music_item O B-artist I-artist O O B-pla...,10
3,3,AddToPlaylist,add this artist to my this is miguel bosé play...,O O B-music_item O B-playlist_owner B-playlist...,10
4,4,AddToPlaylist,add heresy and the hotel choir to the evening ...,O B-entity_name I-entity_name I-entity_name I-...,11
...,...,...,...,...,...
13079,13079,SearchScreeningEvent,find a Consolidated Theatres showing The Good ...,O O B-location_name I-location_name O B-movie_...,10
13080,13080,SearchScreeningEvent,where can i see animated movies in the neighbo...,O O O O B-movie_type I-movie_type B-spatial_re...,9
13081,13081,SearchScreeningEvent,Showtimes for animated movies in the area .,O O B-movie_type I-movie_type B-spatial_relati...,8
13082,13082,SearchScreeningEvent,Which animated movies are playing at Megaplex ...,O B-movie_type I-movie_type O O O B-location_n...,11


In [63]:
a = 'Add Don and Sherri to my Meditate to Sounds of Nature playlist'
b = 'O B-entity_name I-entity_name I-entity_name O B-playlist_owner B-playlist I-playlist I-playlist I-playlist I-playlist O'

In [64]:
class DatasetsTools():
    def __init__(self):
        pass
    
    def csv_line2yaml_line(self, a, b):
        """
        Args:
            a: str, 'Add Don and Sherri to my Meditate to Sounds of Nature playlist'
            b: str, 'O B-entity_name I-entity_name I-entity_name O B-playlist_owner B-playlist I-playlist I-playlist I-playlist I-playlist O'
        Out:
            str, 'Add [entity_name](Don and Sherri) to [playlist_owner](my) [playlist](Meditate to Sounds of Nature) playlist'
        """
        a = a.split()
        b = b.split()
        sen = []
        tag_name = ''
        tag_seq = {}
        for i in range(len(b)):
            tag = b[i]
            word = a[i]
            if tag == 'O':
                if len(tag_seq) != 0:
                    sen.append(f"[{tag_name}]" + '(' + ' '.join(tag_seq[tag_name]) + ')')
                    tag_seq = {}
                sen.append(word)
            else:
                tag_name_now = tag.replace('B-', '').replace('I-', '')
                if 'B-' in tag:
                    if len(tag_seq) != 0 and tag_name != '':
                        sen.append(f"[{tag_name}]" + '(' + ' '.join(tag_seq[tag_name]) + ')')
                        tag_seq == {}
                    tag_seq.update({tag_name_now : [word]})
                    tag_name = tag_name_now
                elif 'I-' in tag:
        #             print(tag_seq, sen)
                    tag_seq[tag_name_now].append(word)

        if len(tag_seq) != 0:
            sen.append(f"[{tag_name}]" + '(' + ' '.join(tag_seq[tag_name]) + ')')
        sen = ' '.join(sen)
        return sen
    
    def get_from_df_dataset(self, df):
        intents = {}
        tags = {}
        err_cnt = 0
        for (i, r) in df.iterrows():
            intent_name = r['intent_label']
            try:
                line = self.csv_line2yaml_line(r['words'], r['word_labels'])
                line_tags = self.read_tags([line])[0]
                if intent_name in intents.keys():
                    intents[intent_name].append(line)
                    tags[intent_name].append(line_tags)
                else:
                    intents.update({intent_name : [line]})
                    tags.update({intent_name : [line_tags]})
            except IndexError:
                err_cnt += 1
                pass
        return intents, tags

    def read_tags(self, raw_intents):
        """
        Extract tags from yaml line
        """
        intents = []
        for i in raw_intents:
            tag_class_ids = [j.span() for j in re.finditer(r'\[(.*?)\]', i)]
            tag_class, tag_names = [], []
            for t in tag_class_ids:
                tag_class.append(i[t[0]:t[1]].replace('[', '').replace(']', ''))
            i = re.sub(r'\[(.*?)\]', '', i)
            tag_ids = [j.span() for j in re.finditer(r'\((.*?)\)', i)]
            for t in tag_ids:
                tag_names.append(i[t[0]:t[1]].replace('(', '').replace(')', ''))
            tag_map = list(zip(tag_class, tag_names))
            intents.append(tag_map)
        return intents
    
    def get_all_intent_names(self, dset_list):
        names = []
        for i in dset_list:
            names.append(i['name'])
        return list(set(names))
    
    def find_by_intent(self, dset_list):
        pass
    
    def append_intent2yaml(self, extracted_intents, yaml_dset, task='train'):
        for intent in extracted_intents.keys():
            if not intent in get_all_intent_names(yaml_dset['intents']):
                intent_frame = {'type' : 'intent', 'task' : task, \
                               'name' : intent, 'utterances' : extracted_intents[intent]}
                yaml_dset['intents'].append(intent_frame)
            else:
                pass
        return yaml_dset
    
    def append_tag2yaml(self, extracted_entities, yaml_dset):
        tag_by_intents = {}
        for intent in tags.keys():
            tag_by_intents.update({intent : {} })
            tag_list = list(itertools.chain.from_iterable(tags[intent]))
            for pair in tag_list:
                tag_name, tag = pair
                if tag_name in tag_by_intents[intent].keys():
                    tag_by_intents[intent][tag_name].append(tag)
                else:
                    tag_by_intents[intent].update({tag_name : [tag]})
            for tag_name in tag_by_intents[intent].keys():
                tag_by_intents[intent][tag_name] = list(set(tag_by_intents[intent][tag_name]))
        tag_by_intents
    
    def dataset2yaml(self, from_path, to_path='data/nlu_data/custom/dataset.yaml'):
        from_path = os.path.join(ROOT_DIR, from_path)
        yaml_path = os.path.join(ROOT_DIR, to_path)
        file = open(yaml_path)
        dset = yaml.load(file, Loader=yaml.FullLoader)
        train_path = os.path.join(from_path, 'train.csv')
        intents_train, tags_train = self.get_from_df_dataset(pd.read_csv(train_path))
        dset = self.append_intent2yaml(intents_train, dset, 'train')
        if 'valid.csv' in os.listdir(from_path):
            intents_valid, tags_valid = self.get_from_df_dataset(pd.read_csv(os.path.join(from_path, 'valid.csv')))
            dset = self.append_intent2yaml(intents_train, dset, 'valid')
        file = open(yaml_path, 'w')
        dset = yaml.dump(dset,file, sort_keys=True)
            

In [43]:
import itertools

In [65]:
d = DatasetsTools()
d.dataset2yaml('data/nlu_data/snipsai')

In [9]:
intents = {}
tags = {}
err_cnt = 0
for (i, r) in snips.iterrows():
    intent_name = r['intent_label']
    try:
        line = d.csv_line2yaml_line(r['words'], r['word_labels'])
        line_tags = d.read_tags([line])[0]
        if intent_name in intents.keys():
            intents[intent_name].append(line)
            tags[intent_name].append(line_tags)
        else:
            intents.update({intent_name : [line]})
            tags.update({intent_name : [line_tags]})
    except IndexError:
        err_cnt += 1
        pass
print(err_cnt)

11


In [17]:
import yaml
yaml_path = os.path.join(ROOT_DIR, 'data/nlu_data/custom/dataset.yaml')
file = open(yaml_path)

dset = yaml.load(file, Loader=yaml.FullLoader)

In [24]:
def get_all_intent_names(dset_list):
    names = []
    for i in dset_list:
        names.append(i['name'])
    return list(set(names))

In [26]:
for intent in intents.keys():
    if not intent in get_all_intent_names(dset['intents']):
        intent_frame = {'type' : 'intent', 'task' : 'train', \
                       'name' : intent, 'utterances' : intents[intent]}
        dset['intents'].append(intent_frame)

In [27]:
dset['intents']

[{'type': 'intent',
  'task': 'train',
  'name': 'turnLightOff',
  'utterances': ['turn off the [location:room](bathroom) lights',
   'can you turn off the lights in the [location:room](garden) ?',
   'disable the illumination in the [location:room](kitchen) please',
   'shutdown the illumination in [location:room](bedroom)',
   'system , turn off the lightning in the [location:room](toilet)',
   'turn off the lights at the [location:room](stairs)',
   'can you turn off the light [location:current](here)',
   'turn off the lights [location:where_room](next door)',
   "swtich the [location:room](lounge)'s lights off please",
   '[location:room](bedroom) , lights off',
   'i command you to turn off the lights in the [location:room](toilet)',
   'system, light off',
   'off the lights in plocation:room](kitchen)',
   'disable light at [location:room](home)',
   'disable lights in [location:room](kitchen) please',
   'disable this lights in the [location:room](yard)',
   'disable lights in

In [117]:
intents.keys()

dict_keys(['AddToPlaylist', 'BookRestaurant', 'GetWeather', 'PlayMusic', 'RateBook', 'SearchCreativeWork', 'SearchScreeningEvent'])

In [112]:
intents['AddToPlaylist'][0], tags['AddToPlaylist'][0]

('Add [entity_name](Don and Sherri) to [playlist_owner](my) [playlist](Meditate to Sounds of Nature) playlist',
 [('entity_name', 'Don and Sherri'),
  ('playlist_owner', 'my'),
  ('playlist', 'Meditate to Sounds of Nature')])